In [1]:
#encoding: utf-8
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
import pickle
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [2]:
FEATURE_TXT = './files/feature.txt'
FILES_PATH = './files'
DOT_MV = '.mv'

In [3]:
def get_column_names():
    column_names = []
    with open(FEATURE_TXT, 'r') as f:
        for line in f:
            column_name = line.strip().split(':')[0]
            column_names.append(column_name)
    return column_names

In [4]:
def get_dfs():
    filenames = os.listdir(FILES_PATH)
    mv_filenames = []
    for filename in filenames:
        if DOT_MV in filename:
            mv_filenames.append(filename)

    mv_dfs = []
    for filename in mv_filenames:
        df = pd.read_csv(os.path.join(FILES_PATH, filename), sep=' ', header=None, names=column_names, na_values='?', index_col=False)
        mv_dfs.append(df)
    
    return mv_dfs

In [5]:
column_names = get_column_names()
mv_dfs = get_dfs()
mv_df = mv_dfs[0].append(mv_dfs[1:])

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
mv_df.head(100)

,C1,C2,D1,C3,C4,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20,D21,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D22,D23,D24,D25,D26,C15,C16,D27,D28,D29,D30,D31,D32,D33,D34,D35,D36,D37,D38,D39,D40,C17,C18,C19,C20,C21,C22,C23,C24,D41,D42,D43,D44,C25,C26,C27,C28,D45,C29,C30,D46,D47,D48,D49,D50,D51,D52,D53,D54,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,class
0,1997,124,4.0,39.9847,-74.0687,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,4.0,4.0,NaN,1.0,NaN,NaN,2.0,0,1,0,0,0,0,0,0,0,1.0,2189.2,0.995082,0.000820,0.000820,0.001639,0.000000,0.000000,0.001639,0.015574,0.472951,0.527049,0.026230,0.068852,0.017213,0.047541,0.077049,0.123770,0.250000,0.389344,58.5,57.4,60.0,1161.0200,1.89,0.160742,0.221020,0.091190,0.414219,0.003091,0.012365,376,2.43,0.702938,0.277778,0.019284,3908.3400,1.14194,3,3,3,3,3,3,5,6,6,6,6,6,6,6,6,6,6,6,7,7,7,6,6,6,6,6,7,4,5,6,0,6,5,5,5,6,5,4,3,6,6,6,3,4,2,2,2,2,5,1,1,1,1,1,1,1,9,3,3,3,4,5,3,2,4,2,3,4,4,4,3,5,6,6,2,2,2,1,1,2,1,3,3,0
1,2000,124,3.0,42.2892,-71.3546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,NaN,NaN,NaN,3.0,2.0,0,1,0,0,0,0,0,0,0,48.0,3596.0,0.883452,0.030249,0.002669,0.036477,0.001779,0.024021,0.021352,0.048043,0.480427,0.519573,0.065836,0.133452,0.034697,0.163701,0.221530,0.164591,0.129004,0.087189,34.3,34.8,33.6,1593.2400,2.26,0.168675,0.158635,0.192771,0.232932,0.006024,0.042169,273,2.96,0.033010,0.403883,0.563107,1647.6300,47.70060,3,3,4,3,4,4,7,6,6,6,6,6,6,6,6,6,6,6,7,7,7,6,6,6,6,7,7,6,6,6,6,6,5,6,5,4,4,3,3,5,5,4,6,5,3,2,2,4,7,3,3,2,1,1,1,1,9,5,3,3,4,4,2,2,4,2,2,3,4,4,2,4,5,5,2,3,3,2,3,3,3,3,3,1
2,1999,141,2.0,40.9518,-72.9342,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,4.0,4.0,3.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,6.0,1.0,2.0,4.0,1.0,2.0,1.0,0.0,1.0,0.0,4.0,4.0,NaN,NaN,NaN,NaN,2.0,0,1,0,0,0,0,0,0,0,45.0,2686.9,0.964015,0.015151,0.000000,0.014204,0.000000,0.001894,0.004735,0.059659,0.518939,0.481061,0.103220,0.220644,0.034091,0.122159,0.214962,0.160038,0.094697,0.050189,31.1,30.8,31.4,860.0070,3.09,0.106509,0.071006,0.378698,0.177515,0.035503,0.056213,251,3.49,0.091398,0.706989,0.201613,946.5170,43.18350,3,3,3,3,3,3,5,6,6,6,6,6,6,6,6,6,6,6,7,7,7,6,6,6,6,6,7,5,6,6,6,6,6,6,5,5,5,4,3,5,6,5,4,4,3,2,2,3,6,1,2,1,1,1,1,1,9,3,3,3,4,5,2,2,4,2,2,3,4,4,3,5,6,6,2,3,2,2,2,2,2,4,4,0
3,2003,120,4.0,43.3035,-70.5973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,3.0,4.0,NaN,NaN,NaN,NaN,1.0,0,1,0,0,0,0,0,0,0,30.0,315.3,0.978050,0.001291,0.004519,0.003228,0.000000,0.000000,0.012912,0.004519,0.477082,0.522918,0.053583,0.140736,0.026469,0.066495,0.125242,0.157521,0.215623,0.214332,45.6,43.8,46.9,144.7320,2.18,0.113924,0.212377,0.184248,0.354430,0.011252,0.028129,440,2.77,0.391267,0.487158,0.121575,237.7590,27.52010,3,3,4,3,4,3,9,6,6,6,6,7,6,6,7,6,7,7,8,7,7,6,6,7,7,7,7,6,5,5,5,5,4,5,4,

In [8]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 50)

In [9]:
# 协方差
# correlations = mv_df.corr()
# ser = correlations.loc['class']
# print(ser)

In [10]:
# 查看有哪些列有太多的缺失值
# many_missing_columns = []
# for column_name in mv_df.columns.tolist():
#     print('{}:'.format(column_name), mv_df[mv_df[column_name].isnull()==True].shape[0])
#     if mv_df[mv_df[column_name].isnull()==True].shape[0] > 2000:
#         many_missing_columns.append(column_name)
# print(len(many_missing_columns))

In [11]:
# 删掉这些列
# for column_name in many_missing_columns:
#     del mv_df[column_name]

In [12]:
# 以下列的协方差为 0，删除这些列
# del_columns = ['D5', 'D50', 'D51', 'D52', 'C81', 'C121', 'C122', 'C123']
# for column_name in del_columns:
#     if column_name not in many_missing_columns:  # 判断是否已经删除过
#         del mv_df[column_name]

# 更新列名
# column_names = mv_df.columns.tolist()

# check
# print('many_missing_columns:', many_missing_columns)
# print('column num:', len(column_names))

In [13]:
conv_names = ['C{}'.format(i) for i in range(1, 5)] + ['C{}'.format(i) for i in range(31, 69)]
disc_names = [column_name for column_name in column_names if column_name not in conv_names]
disc_names.remove('class')

# check
print('conv column num:', len(conv_names))
print('disc column num:', len(disc_names))
print('total column num:', len(conv_names) + len(disc_names))

conv column num: 42
disc column num: 163
total column num: 205


In [14]:
mv_conv_df = mv_df[conv_names]
mv_disc_df = mv_df[disc_names]
labels = mv_df['class']

# 查看两个类别各有多少个标签
# print(labels.value_counts())

# check
print('mv_conv_df shape:', mv_conv_df.shape)
print('mv_disc_df shape:', mv_disc_df.shape)
print('labels shape:', labels.shape)


# 重置索引
mv_conv_df = mv_conv_df.reset_index(drop=True)
mv_disc_df = mv_disc_df.reset_index(drop=True)


# 填补缺失值
for column_name in mv_conv_df.columns.tolist():
    mean_val = mv_conv_df[column_name].mean()
    mv_conv_df[column_name].fillna(mean_val, inplace=True)
mv_disc_df = mv_disc_df.fillna(10000)

# mv_disc_df：众数填补缺失值
# for column_name in mv_disc_df.columns.tolist():
#     fill_val = mv_disc_df[column_name].dropna().mode()[0]
#     mv_disc_df[column_name].fillna(fill_val, inplace=True)

# mv_disc_df：基于每条数据的连续值特征（连续值特征缺失值已经填补了）使用 knn 算法填补缺失值
# mv_disc_df = mv_disc_df.fillna(10000)
# nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(mv_conv_df)  # 根据非连续特征选取 10 条最相近的数据
# distances, indices = nbrs.kneighbors(mv_conv_df)
# for column_name in mv_disc_df.columns.tolist():
#     list_temp = []
#     for idx, x in enumerate(mv_disc_df[column_name]):
#         if x == 10000:  # 有缺失值
#             values = []
#             for i in indices[idx]:
#                 if mv_disc_df[column_name][i] != 10000:
#                     values.append(int(mv_disc_df[column_name][i]))
#             if len(values) != 0:
#                 counts = np.bincount(np.array(values))
#                 value = np.argmax(counts)
#                 list_temp.append(value)
#             else:
#                 list_temp.append(10000)
#         else:
#             list_temp.append(x)
#     mv_disc_df[column_name] = list_temp

# 数据类型转换
for column_name in mv_conv_df.columns.tolist():
    mv_conv_df[column_name] = mv_conv_df[column_name].astype(np.float64)
for column_name in mv_disc_df.columns.tolist():
    mv_disc_df[column_name] = mv_disc_df[column_name].astype(np.int32)
    
# check
print(mv_conv_df.isnull().any()[mv_conv_df.isnull().any() == True])
print(mv_disc_df.isnull().any()[mv_disc_df.isnull().any() == True])

mv_conv_df shape: (9000, 42)
mv_disc_df shape: (9000, 163)
labels shape: (9000,)
Series([], dtype: bool)
Series([], dtype: bool)


In [15]:
enc = OneHotEncoder()
enc.fit(mv_disc_df)
disc_array = enc.transform(mv_disc_df).toarray()

# 连续型特征归一化
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(mv_conv_df)
conv_array = min_max_scaler.transform(mv_conv_df)

# 连续型特征标准化
# standard_scaler = StandardScaler()
# standard_scaler.fit(mv_conv_df)
# conv_array = standard_scaler.transform(mv_conv_df)

# check
print('disc array shape:', disc_array.shape)
print('conv array shape:', conv_array.shape)

disc array shape: (9000, 665)
conv array shape: (9000, 42)


In [16]:
# 得到训练数据
X_data = np.hstack((conv_array, disc_array))
y_data = labels

# check
print('X_data shape:', X_data.shape)
print('y_data shape:', y_data.shape)

# 数据集按顺序划分成 5 份（与之前的 5 份是相同的）
X_data_list = np.split(X_data, 5)
y_data_list = np.split(y_data, 5)

# check
print('X_data_list len:', len(X_data_list))
print('y_data_list len:', len(y_data_list))
for i in range(len(X_data_list)):
    print('X_data_list[{}] shape:'.format(i), X_data_list[i].shape)
    print('y_data_list[{}] shape:'.format(i), y_data_list[i].shape)

X_data shape: (9000, 707)
y_data shape: (9000,)
X_data_list len: 5
y_data_list len: 5
X_data_list[0] shape: (1800, 707)
y_data_list[0] shape: (1800,)
X_data_list[1] shape: (1800, 707)
y_data_list[1] shape: (1800,)
X_data_list[2] shape: (1800, 707)
y_data_list[2] shape: (1800,)
X_data_list[3] shape: (1800, 707)
y_data_list[3] shape: (1800,)
X_data_list[4] shape: (1800, 707)
y_data_list[4] shape: (1800,)


In [17]:
# PCA 降维
pca = PCA(n_components=5)
pca.fit(X_data)
X_data_pca = pca.transform(X_data)

# 数据集按顺序划分成 5 份（与之前的 5 份是相同的）
X_data_pca_list = np.split(X_data_pca, 5)
y_data_pca_list = np.split(y_data, 5)

# check
print('X_data_pca_list len:', len(X_data_pca_list))
print('y_data_pca_list len:', len(y_data_pca_list))
for i in range(len(X_data_pca_list)):
    print('X_data_pca_list[{}] shape:'.format(i), X_data_pca_list[i].shape)
    print('y_data_pca_list[{}] shape:'.format(i), y_data_pca_list[i].shape)

X_data_pca_list len: 5
y_data_pca_list len: 5
X_data_pca_list[0] shape: (1800, 5)
y_data_pca_list[0] shape: (1800,)
X_data_pca_list[1] shape: (1800, 5)
y_data_pca_list[1] shape: (1800,)
X_data_pca_list[2] shape: (1800, 5)
y_data_pca_list[2] shape: (1800,)
X_data_pca_list[3] shape: (1800, 5)
y_data_pca_list[3] shape: (1800,)
X_data_pca_list[4] shape: (1800, 5)
y_data_pca_list[4] shape: (1800,)


In [18]:
def lr_func(X_data_list, y_data_list):
    res = []
    for penalty in ['l1', 'l2']:
        acc_list = []
        y_pred_all = []
        clf = LogisticRegression(penalty=penalty, solver='liblinear')
        for idx, x_data in enumerate(X_data_list):
            X_test = x_data
            y_test = y_data_list[idx]
            X_train = np.vstack(X_data_list[0: idx] + X_data_list[idx + 1: ])
            y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_pred_all.extend(y_pred)
            acc = (y_test == y_pred).mean() * 100
            acc_list.append(acc)
            print('cross validation {}: the accuracy is {:.4f}'.format(idx, acc))
        print('mean accuracy: {:4f}'.format(np.array(acc_list).mean()))
        res.append(y_pred_all)
    return res

In [19]:
def bnb_func(X_data_list, y_data_list):
    res = []
    clf = BernoulliNB()
    acc_list = []
    y_pred_all = []
    for idx, x_data in enumerate(X_data_list):
        X_test = x_data
        y_test = y_data_list[idx]
        X_train = np.vstack(X_data_list[0: idx] + X_data_list[idx + 1: ])
        y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred_all.extend(y_pred)
        acc = (y_test == y_pred).mean() * 100
        acc_list.append(acc)
        print('cross validation {}: the accuracy is {:.4f}'.format(idx, acc))
    print('mean accuracy: {:4f}'.format(np.array(acc_list).mean()))
    res.append(y_pred_all)
    return res

In [20]:
def mnb_func(X_data_list, y_data_list):
    res = []
    clf = MultinomialNB()
    acc_list = []
    y_pred_all = []
    for idx, x_data in enumerate(X_data_list):
        X_test = x_data
        y_test = y_data_list[idx]
        X_train = np.vstack(X_data_list[0: idx] + X_data_list[idx + 1: ])
        y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred_all.extend(y_pred)
        acc = (y_test == y_pred).mean() * 100
        acc_list.append(acc)
        print('cross validation {}: the accuracy is {:.4f}'.format(idx, acc))
    print('mean accuracy: {:4f}'.format(np.array(acc_list).mean()))
    res.append(y_pred_all)
    return res

In [21]:
def knn_func(X_data_list, y_data_list):
    res = []
    for n_neighbors in [1, 3, 5]:
        print('n_neighbors = {}:'.format(n_neighbors))
        clf = KNeighborsClassifier(n_neighbors=n_neighbors)
        acc_list = []
        y_pred_all = []
        for idx, x_data in enumerate(X_data_list):
            X_test = x_data
            y_test = y_data_list[idx]
            X_train = np.vstack(X_data_list[0: idx] + X_data_list[idx + 1: ])
            y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_pred_all.extend(y_pred)
            acc = (y_test == y_pred).mean() * 100
            acc_list.append(acc)
            print('cross validation {}: the accuracy is {:.4f}'.format(idx, acc))
        print('mean accuracy: {:4f}'.format(np.array(acc_list).mean()))
        res.append(y_pred_all)
    return res

In [22]:
def rf_func(X_data_list, y_data_list):
    res = []
    for n_estimators in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100, 200, 300]:
        print('n_estimators = {}:'.format(n_estimators))
        clf = RandomForestClassifier(n_estimators=n_estimators)
        acc_list = []
        y_pred_all = []
        for idx, x_data in enumerate(X_data_list):
            X_test = x_data
            y_test = y_data_list[idx]
            X_train = np.vstack(X_data_list[0: idx] + X_data_list[idx + 1: ])
            y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_pred_all.extend(y_pred)
            acc = (y_test == y_pred).mean() * 100
            acc_list.append(acc)
            print('cross validation {}: the accuracy is {:.4f}'.format(idx, acc))
        print('mean accuracy: {:4f}'.format(np.array(acc_list).mean()))
        res.append(y_pred_all)
    return res

In [23]:
def adaboost_func(X_data_list, y_data_list):
    res = []
    for n_estimators in [5, 10, 15, 20]:
        for algorithm in ['SAMME', 'SAMME.R']:
            clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2, min_samples_split=20, min_samples_leaf=1), algorithm=algorithm, n_estimators=n_estimators, learning_rate=0.8)
            acc_list = []
            y_pred_all = []
            for idx, x_data in enumerate(X_data_list):
                X_test = x_data
                y_test = y_data_list[idx]
                X_train = np.vstack(X_data_list[0: idx] + X_data_list[idx + 1: ])
                y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                y_pred_all.extend(y_pred)
                acc = (y_test == y_pred).mean() * 100
                acc_list.append(acc)
                print('cross validation {}: the accuracy is {:.4f}'.format(idx, acc))
            print('mean accuracy: {:4f}'.format(np.array(acc_list).mean()))
            res.append(y_pred_all)
    return res

In [24]:
def light_gbm_func(X_data_list, y_data_list):
    res = []
    for max_depth in [3, 4, 5, 6, 7, 8, 9]:
        clf = lgb.LGBMClassifier(learning_rate=0.1, max_depth=max_depth, n_estimators=1000, objective = 'binary', reg_lambda=1)
        acc_list = []
        y_pred_all = []
        for idx, x_data in enumerate(X_data_list):
            X_test = x_data
            y_test = y_data_list[idx]
            X_train = np.vstack(X_data_list[0: idx] + X_data_list[idx + 1: ])
            y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_pred_all.extend(y_pred)
            acc = (y_test == y_pred).mean() * 100
            acc_list.append(acc)
            print('cross validation {}: the accuracy is {:.4f}'.format(idx, acc))
        print('mean accuracy: {:.4f}'.format(np.array(acc_list).mean()))
        res.append(y_pred_all)
    return res

In [25]:
def nn_func(X_data_list, y_data_list):
    """
    简单神经网络
    """
    input_dim = X_data_list[0].shape[1]
    res = []
    acc_list = []
    y_pred_all = []
    for idx, x_data in enumerate(X_data_list):
        model = Sequential()
        model.add(Dense(1024, input_dim=input_dim, init='uniform', activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(1, init='uniform', activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        X_test = x_data
        y_test = y_data_list[idx]
        X_train = np.vstack(X_data_list[0: idx] + X_data_list[idx + 1: ])
        y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
        model.fit(X_train, y_train, nb_epoch=150, batch_size=128, verbose=1)
        y_pred = model.predict(X_test)
        y_pred[y_pred >= 0.5] = 1
        y_pred[y_pred < 0.5] = 0
        y_pred = y_pred.reshape(-1,)
        y_pred_all.extend(y_pred)
        acc = (y_test == y_pred).mean() * 100
        acc_list.append(acc)
        print('cross validation {}: the accuracy is {:.4f}'.format(idx, acc))
    print('mean accuracy: {:.4f}'.format(np.array(acc_list).mean()))
    res.append(y_pred_all)
    return res

In [26]:
def load_res(pickle_name, pickle_files, func, X_data_list, y_data_list):
    if pickle_name in pickle_files:
        with open('./tmp/{}'.format(pickle_name), 'rb') as f:
            res = pickle.load(f)
    else:
        res = func(X_data_list, y_data_list)
        with open('./tmp/{}'.format(pickle_name), 'wb') as f:
            pickle.dump(res, f)
    return res

In [27]:
def stacking():
    
    pickle_files = os.listdir('./tmp')
    
    rf_res = load_res('rf.pickle', pickle_files, rf_func, X_data_list, y_data_list)
    knn_res = load_res('knn.pickle', pickle_files, knn_func, X_data_list, y_data_list)
    lr_res = load_res('lr.pickle', pickle_files, lr_func, X_data_list, y_data_list)
    bnb_res = load_res('bnb.pickle', pickle_files, bnb_func, X_data_list, y_data_list)
    mnb_res = load_res('mnb.pickle', pickle_files, mnb_func, X_data_list, y_data_list)
    # adaboost_res = load_res('adaboost.pickle', pickle_files, adaboost_func, X_data_list, y_data_list)  # 引入以后效果会变差
    lgbm_res = load_res('lgbm.pickle', pickle_files, light_gbm_func, X_data_list, y_data_list)
    nn_res = load_res('nn.pickle', pickle_files, nn_func, X_data_list, y_data_list)

    rf_pca_res = load_res('rf.pca.pickle', pickle_files, rf_func, X_data_pca_list, y_data_pca_list)
    knn_pca_res = load_res('knn.pca.pickle', pickle_files, knn_func, X_data_pca_list, y_data_pca_list)
    lr_pca_res = load_res('lr.pca.pickle', pickle_files, lr_func, X_data_pca_list, y_data_pca_list)
    # adaboost_pca_res = load_res('adaboost.pca.pickle', pickle_files, adaboost_func, X_data_pca_list, y_data_pca_list)  # 引入以后效果不提升
    lgbm_pca_res = load_res('lgbm.pca.pickle', pickle_files, light_gbm_func, X_data_pca_list, y_data_pca_list)

    predicts = []
    
    predicts.extend(rf_res)
    predicts.extend(knn_res)
    predicts.extend(lr_res)
    predicts.extend(lgbm_res)
    # predicts.extend(nn_res)

    predicts.extend(rf_pca_res)
    predicts.extend(knn_pca_res)
    predicts.extend(lr_pca_res)
    predicts.extend(lgbm_pca_res)

    predict_data = np.array(predicts).T
    return predict_data

In [28]:
predict_data = stacking()
X_data_augment = np.hstack([predict_data, X_data])
X_data_augment_list = np.split(X_data_augment, 5)

# check
print('X_data_augment shape:', X_data_augment.shape)
for i in range(len(X_data_augment_list)):
    print('X_data_augment_list[{}] shape:'.format(i), X_data_augment_list[i].shape)

X_data_augment shape: (9000, 759)
X_data_augment_list[0] shape: (1800, 759)
X_data_augment_list[1] shape: (1800, 759)
X_data_augment_list[2] shape: (1800, 759)
X_data_augment_list[3] shape: (1800, 759)
X_data_augment_list[4] shape: (1800, 759)


In [29]:
def xgb_stacking():

    predict_data = stacking()
    X_data_augment = np.hstack([predict_data, X_data])
    X_data_augment_list = np.split(X_data_augment, 5)

    # check
    print('X_data_augment shape:', X_data_augment.shape)
    for i in range(len(X_data_augment_list)):
        print('X_data_augment_list[{}] shape:'.format(i), X_data_augment_list[i].shape)
    
    # 定义寻参列表
    # eta_list = [0.01, 0.1, 0.2, 0.3]
    # max_depth_list = [3, 5, 7, 9]
    # min_child_weight_list = [1, 2, 3, 4, 5]
    # gamma_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

    eta_list = [0.1, 0.3]
    max_depth_list = [3]
    min_child_weight_list = [1, 2, 3, 4, 5]
    gamma_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
    
    best_mean_accuracy = 0

    for eta in eta_list:
        for max_depth in max_depth_list:
            for min_child_weight in min_child_weight_list:
                for gamma in gamma_list:
                    # 定义参数
                    params = {
                        'objective': 'binary:logistic',
                        'eta': eta,
                        'max_depth': max_depth,
                        'min_child_weight': min_child_weight,
                        'seed': 0,
                        'class_num': 2,
                        'silent': 1,
                        'gamma': gamma
                    }
                    acc_list = []
                    for idx, x_data in enumerate(X_data_augment_list):
                        X_test = x_data
                        y_test = y_data_list[idx]
                        X_train = np.vstack(X_data_augment_list[0: idx] + X_data_augment_list[idx + 1: ])
                        y_train = np.array(y_data_list[0: idx] + y_data_list[idx + 1: ]).reshape((-1,))
                        xg_train = xgb.DMatrix(X_train, label=y_train)
                        xg_test = xgb.DMatrix(X_test, label=y_test)
                        watchlist = [(xg_train, 'train'), (xg_test, 'test')]
                        bst = xgb.train(params, xg_train, 1000, watchlist, early_stopping_rounds=15, verbose_eval=False)
                        y_pred = bst.predict(xg_test, ntree_limit=bst.best_ntree_limit)
                        y_pred[y_pred >= 0.5] = 1
                        y_pred[y_pred < 0.5] = 0
                        acc = (y_test == y_pred).mean()
                        acc_list.append(acc)
                    mean_accuracy = np.array(acc_list).mean()
                    print('eta {}, max_depth {}, min_child_weight {}, gamma {}, acc_list {}, mean accuracy {:.5f}'.format(eta, max_depth, min_child_weight, gamma, acc_list, mean_accuracy))
                    if mean_accuracy > best_mean_accuracy:
                        best_mean_accuracy = mean_accuracy
    print('best mean accuracy: {:.5f}'.format(best_mean_accuracy))

In [30]:
xgb_stacking()

X_data_augment shape: (9000, 759)
X_data_augment_list[0] shape: (1800, 759)
X_data_augment_list[1] shape: (1800, 759)
X_data_augment_list[2] shape: (1800, 759)
X_data_augment_list[3] shape: (1800, 759)
X_data_augment_list[4] shape: (1800, 759)
eta 0.1, max_depth 3, min_child_weight 1, gamma 0, acc_list [0.77, 0.7877777777777778, 0.7855555555555556, 0.7816666666666666, 0.7788888888888889], mean accuracy 0.78078
eta 0.1, max_depth 3, min_child_weight 1, gamma 0.1, acc_list [0.77, 0.7877777777777778, 0.7855555555555556, 0.7816666666666666, 0.7788888888888889], mean accuracy 0.78078
eta 0.1, max_depth 3, min_child_weight 1, gamma 0.2, acc_list [0.77, 0.7877777777777778, 0.7855555555555556, 0.7816666666666666, 0.7788888888888889], mean accuracy 0.78078
eta 0.1, max_depth 3, min_child_weight 1, gamma 0.3, acc_list [0.77, 0.7877777777777778, 0.7855555555555556, 0.7816666666666666, 0.7788888888888889], mean accuracy 0.78078
eta 0.1, max_depth 3, min_child_weight 1, gamma 0.4, acc_list [0.77, 0

eta 0.3, max_depth 3, min_child_weight 3, gamma 0.5, acc_list [0.7677777777777778, 0.7811111111111111, 0.7833333333333333, 0.78, 0.7794444444444445], mean accuracy 0.77833
eta 0.3, max_depth 3, min_child_weight 4, gamma 0, acc_list [0.7666666666666667, 0.78, 0.7833333333333333, 0.78, 0.7783333333333333], mean accuracy 0.77767
eta 0.3, max_depth 3, min_child_weight 4, gamma 0.1, acc_list [0.7666666666666667, 0.78, 0.7833333333333333, 0.78, 0.7783333333333333], mean accuracy 0.77767
eta 0.3, max_depth 3, min_child_weight 4, gamma 0.2, acc_list [0.7666666666666667, 0.78, 0.7833333333333333, 0.78, 0.7783333333333333], mean accuracy 0.77767
eta 0.3, max_depth 3, min_child_weight 4, gamma 0.3, acc_list [0.7666666666666667, 0.78, 0.7833333333333333, 0.78, 0.7783333333333333], mean accuracy 0.77767
eta 0.3, max_depth 3, min_child_weight 4, gamma 0.4, acc_list [0.7666666666666667, 0.78, 0.7833333333333333, 0.78, 0.7783333333333333], mean accuracy 0.77767
eta 0.3, max_depth 3, min_child_weight 4